In [41]:
import pandas as pd

all=pd.read_csv("imdb.csv",encoding='unicode_escape')
all=all.loc[all['label']!="unsup"]

X_test=(all.loc[all['type']=='test']).loc[:,['review']]
y_test=(all.loc[all['type']=='test']).loc[:,['label']]

X_train=(all.loc[all['type']=='train']).loc[:,['review']]
y_train=(all.loc[all['type']=='train']).loc[:,['label']]


In [42]:
#tokenizing reviews
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~`'''
test_reviews=X_test.to_dict()
train_reviews=X_train.to_dict()

#print(test_reviews['review'])
def tokenizing(rev):
 tokens = nltk.word_tokenize(rev)
 stop_words = set(stopwords.words('english'))
 tokens_l = [w.lower() for w in tokens]
 token_final=[]
 for item in tokens_l : 
  item= item.strip(punc)
  if item not in stop_words and item !="":
   token_final.append(item)
 text = nltk.Text(tokens)
 final_text=""
 for wd in token_final:
  final_text=final_text+wd+" " 
 #only_nn = [x for (x,y) in token_final if y in ('NN')]
 #match = text.concordance('language')
 return [final_text, token_final]

actual_test_text=dict()
for x in test_reviews['review']:
 #print(x)
 #print(test_reviews['review'][x])
 actual_review=test_reviews['review'][x]
 txt=tokenizing(actual_review)[0]
 actual_test_text[x]=txt

actual_train_text=dict()
for x in train_reviews['review']:
 #print(x)
 #print(test_reviews['review'][x])
 actual_review=train_reviews['review'][x]
 txt=tokenizing(actual_review)[0]
 actual_train_text[x]=txt

 #print(actual_test)

In [43]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_test["label"])
y_test=le.transform(y_test["label"])

le.fit(y_train["label"])
y_train=le.transform(y_train["label"])

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer# Create feature vectors
df_test_text=pd.DataFrame.from_dict(actual_test_text, orient="index", columns=["review"])
df_train_text=pd.DataFrame.from_dict(actual_train_text, orient="index", columns=["review"])
#print(df_test_text.all)

vectorizer = TfidfVectorizer(max_features=6000,
                             min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
x_train_vectors = vectorizer.fit_transform(df_train_text['review']).toarray()
x_test_vectors = vectorizer.transform(df_test_text['review']).toarray()


In [55]:
print(x_train_vectors)
print(y_train)
print(x_test_vectors)
print(y_test)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.04614983 ... 0.         0.         0.09679047]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[0 0 0 ... 1 1 1]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.06081249 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[0 0 0 ... 1 1 1]


In [82]:
from sklearn import svm

clf = svm.LinearSVC(C=0.1)
clf.fit(x_train_vectors, y_train)

LinearSVC(C=0.1)

In [84]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pred=clf.predict(x_test_vectors)
print(confusion_matrix(y_test,pred))  
print(classification_report(y_test,pred))  
print(accuracy_score(y_test, pred))

[[10994  1506]
 [ 1361 11139]]
              precision    recall  f1-score   support

           0       0.89      0.88      0.88     12500
           1       0.88      0.89      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000

0.88532


In [110]:
tweet="I love Andrew Garfield and this was the main motif, for me, to see this film. And real, he did a great role. And , real, my appreciation for him becomes more intense. And the admiration for great job of Lin Manuel Miranda. The film can be seen as hommage to Jonathan Larson. In same measure, it works very well as nice sketch for many anonimous lives, not very different by the lead character, between love, friendship, self acceptance, ambitious projects and need to be remarked as fair result of not doubts vocation. And a nice kick to they far to have opportunity of courage to become a real name. That last thing , the mix of confrontations, desires and dreams and frustrations , well used, well served represents the great good point of this movie. And, sure, fine exploration of the universe of shows and expressions of pure freedom. It can be perceived as a sketch about truth in its complex nuances , off course, but, in same measure, it is a magnificent musical, impressive not only for familiar ingredients of genre, for admirable energy of Andrew Garfield, for beautiful acting for supporting roles , but for high honesty. It is a very, very, very honest film, thing so rare today. And that transforms it in a personal story, easy as kick to remind the transition from 29 years old to 30, the round crisis, the decision to begin real life. Few scenes in this film are just gorgeous. So, must see it ! Not for be seduced. But for a great, profound useful perspective about life."

tok = tokenizing(tweet)[0]
x=vectorizer.transform([tok]).toarray()
result=clf.predict(x)
if result[0]== 0:
 sentiment="negative"
else:
 sentiment="positive"

print(sentiment)

positive
